In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
# chattem:template을 message로부터 만듬 / prompttem:string을 이용해 template만듬

chat = ChatOpenAI(temperature=0.1)
# b = chat.predict("How many planets are there?")

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

In [ ]:
# 직접 message를 지정해서 실행. 사용할 라이브러리 다 불러와야 함.

from langchain.schema import HumanMessage, AIMessage, SystemMessage
# System:역할지정/ AI:ai 시작할 때 할말/ Human:물어볼 거.

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, What is your name?")
]

chat.predict_messages(messages)

In [ ]:
# chatprompttemplate 사용해서 더 편하게 template 작성
# 커스텀 하기 더 쉬어짐.
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}"),
        ("human", "What is the distance between {country_a} and {country_b}. Also, What is your name?")
    ]
)

prompt = template.from_messages(
    language="Greak",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

## LangChain

### Output Parser : 모델의 출력 처리, 결과를 원하는 형식으로 변환하는 역할.

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

In [2]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",") #공백 제거, ','기준으로 잘라 array저장.
        return list(map(str.strip,items)) #각 단어의 공백을 제거 후 list 저장.
    
#p = CommaOutputParser()
#p.parse("Hello,how, are, you")

['Hello', 'how', 'are', 'you']

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system",
     "You are a list generation machine. Everything you are asked will be"
     "answered with a comma seperated list of max {max_items}. Do NOT reply with anything else.",
    ),
    ("human", "{question}")
])

'''
prompt=template.format_messages(
    max_items=10,
    question="What are the planets"
)

# AIMessage(content='Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune, Pluto')
result = chat.predict_messages(prompt)
p = CommaOutputParser()

p.parse(result.content) # result에서 content 값만 list만듬
'''

In [ ]:
# '|'을 사용해 chain 형성.
# 순서대로 불러와서 처리해줌. chain끼리도 결합 가능.
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items":5,
    "question":"What are the pokemons"
})

## Chain + Chain

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# tem:창의성, str:응답 실시간으로 보기. call:문자 console에 print
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

# chain1 : 레시피 말하기
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow\
    recipies for any type of cuisine with easy to find ingredients. "),
    ("human", "I want to cook {cuisine} food."),
])
chef_chain = chef_prompt | chat

# chain2 : 레시피를 vegetarian 레시피로 변환. 레시피는 바꾸지 않고 재료만
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies\
    vegetarian. You find alternative ingredients and explain their preparation. You\
     say you don't know how to replace it."),
    ("human", "{recipe}")
])
veg_chain = veg_chef_prompt | chat


# chain1값을 chain2입력으로 넣는 작업.
# chain2는 recipe값이 필요하기에 다음과 같이 작성함.
final_chain = {"recipe" : chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

AIMessage(content="Thank you for sharing the recipe for Chicken Tikka Masala! As a vegetarian chef, I specialize in making traditional recipes vegetarian-friendly. In this case, the main ingredient is chicken, which I would replace with a plant-based alternative such as tofu or seitan.\n\nFor this recipe, you can substitute the chicken with extra-firm tofu or seitan. Here's how you can prepare them as a replacement:\n\n1. **Tofu**: \n   - Use extra-firm tofu and press it to remove excess water. Cut the tofu into bite-sized cubes.\n   - To add flavor, you can marinate the tofu in a mixture of soy sauce, garlic powder, ginger, and a touch of oil for at least 30 minutes.\n   - After marinating, you can either pan-fry or bake the tofu until it is golden and slightly crispy.\n\n2. **Seitan**:\n   - Seitan is a high-protein meat substitute made from wheat gluten. You can find it pre-made in many grocery stores or make your own.\n   - If using pre-made seitan, simply cut it into bite-sized pi